<a href="https://colab.research.google.com/github/WojciechMierzwa/Sztuczna_Inteligencja/blob/main/AI_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [32]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

In [33]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

In [34]:
BUFFER_SIZE = 1000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [35]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape, validation_targets.shape)

(6000, 28, 28, 1) (6000,)


In [36]:
input_size = 784
output_size = 10

hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [37]:
NUM_EPOCHS = 30

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_data,
          epochs = NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 1
          )

Epoch 1/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7819 - loss: 0.7418 - val_accuracy: 0.9367 - val_loss: 0.2223
Epoch 2/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9428 - loss: 0.1971 - val_accuracy: 0.9510 - val_loss: 0.1615
Epoch 3/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9574 - loss: 0.1452 - val_accuracy: 0.9608 - val_loss: 0.1333
Epoch 4/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9667 - loss: 0.1160 - val_accuracy: 0.9605 - val_loss: 0.1287
Epoch 5/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9722 - loss: 0.0932 - val_accuracy: 0.9657 - val_loss: 0.1170
Epoch 6/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9769 - loss: 0.0785 - val_accuracy: 0.9675 - val_loss: 0.1114
Epoch 7/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.9801 - loss: 0.0677 - val_accuracy: 0.9658 - val_loss: 0.1186
Epoch 8/30
540/540 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9814 - loss: 0.0626 - val_accuracy

In [38]:
test_loss, test_accuracy = model.evaluate(test_data)

print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 901ms/step - accuracy: 0.9690 - loss: 0.1072
Test loss: 0.11. Test accuracy: 96.90%


In [53]:
import cv2
import numpy as np

for i in range(10):
    img_path = f'/content/{i}.png'
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"Nie znaleziono pliku: {img_path}")
        continue

    img_resized = cv2.resize(img, (28, 28))
    img_normalized = img_resized / 255.0
    img_reshaped = img_normalized.reshape(1, 28, 28, 1)

    prediction = model.predict(img_reshaped, verbose=0)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction) * 100

    print(f"Plik: {i}.png -> Przewidywana cyfra: {predicted_class} z pewnością {confidence:.2f}%")


Plik: 0.png -> Przewidywana cyfra: 0 z pewnością 76.48%
Plik: 1.png -> Przewidywana cyfra: 1 z pewnością 99.16%
Plik: 2.png -> Przewidywana cyfra: 2 z pewnością 82.16%
Plik: 3.png -> Przewidywana cyfra: 3 z pewnością 51.78%
Plik: 4.png -> Przewidywana cyfra: 5 z pewnością 68.27%
Plik: 5.png -> Przewidywana cyfra: 5 z pewnością 95.76%
Plik: 6.png -> Przewidywana cyfra: 6 z pewnością 33.83%
Plik: 7.png -> Przewidywana cyfra: 3 z pewnością 99.83%
Plik: 8.png -> Przewidywana cyfra: 3 z pewnością 87.75%
Plik: 9.png -> Przewidywana cyfra: 3 z pewnością 97.23%
